# Libraries

In [1]:
import ast
import gensim
import gensim.corpora as corpora 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import scipy.cluster.hierarchy as shc

from collections import Counter
from gensim.models import CoherenceModel
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.neighbors._nearest_centroid import NearestCentroid

# Data

In [2]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/220120_feature_matrix_no_bert.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))


/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (361,363,364,366,367,375,379,384,389,394,399,400,401,402) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Feature Matrix Curation

## Processing Metadata

In [3]:
loc = dat_obesity.columns.get_loc
dat_obesity_4 = dat_obesity.iloc[:,np.r_[loc("Likes at Posting"):loc("Followers at Posting")+1, 
                                         loc("Total Interactions"):loc("Care") + 1, 
                                         loc("Total Interactions (weighted  —  Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )")]]
dat_obesity_4 = dat_obesity_4.replace(' ', '', regex=True)    
dat_obesity_4 = dat_obesity_4.fillna(0)                                    

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
loc = dat_obesity.columns.get_loc

dat_obesity_1 = dat_obesity.loc[:,"abnormal hormone":"years old"]
dat_obesity_2 = dat_obesity.loc[:,"neg":"compound"]
dat_obesity_3 = dat_obesity.loc[:,"WC":"OtherP"]
dat_obesity_fm = pd.concat([dat_obesity_1,
                            dat_obesity_2,
                            dat_obesity_3,
                            dat_obesity_4],
                            axis = 1)
print(dat_obesity_fm.shape) #522,473            


/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(522473, 463)


# Clustering

- Randomly Sample and Run Hierarchical Clustering, then run KMeans and DBSCAN
- LDA
- Pick Features Based on Reddit Analysis

## LDA Clustering

### Initializing Terms and Texts

In [5]:
texts = [ast.literal_eval(x) for x in dat_obesity.tokens]

terms = [item for sublist in texts for item in sublist]
terms = list(set(terms)) #106269

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Initial LDA Set-Up

In [6]:
# Create Dictionary
id2word = corpora.Dictionary(texts) 

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts] 

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Picking Best # of Topics

In [ ]:
# number of topics
for i in range(2, 26):
    lda_model_loop = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i,
                                           random_state = 110295)

    coherence_model_lda_loop = CoherenceModel(model=lda_model_loop, 
                                         texts=texts, 
                                         dictionary=id2word, 
                                         coherence='c_v')
    
    coherence_lda_loop = coherence_model_lda_loop.get_coherence()                                       
    print(f"Coherence of {coherence_lda_loop} for {i} topics.")

### Visualization

In [7]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14,
                                           random_state = 110295)

#vis = pyLDAvis.gensim_models.prepare(topic_model = lda_model, 
#                       corpus = corpus, 
#                       dictionary = id2word)
#pyLDAvis.enable_notebook()
#pyLDAvis.display(vis)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Export

In [39]:
#pyLDAvis.save_html(vis, 'lda.html')

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Exporting Clusters

In [8]:
topic_probabilities = []
[topic_probabilities.append(i) for i in lda_model[corpus]]

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [9]:
topic_probs_dict = {i: topic_probabilities[i] for i in range(0, len(topic_probabilities))}

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
test = pd.DataFrame.from_dict({k:dict(v) for k,v in topic_probs_dict.items()}, orient='index').reset_index()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
test.sort_values('index').to_csv(file_path + "220125_lda_topic_probabilities.csv")

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
dat_obesity["tokens_bigrams"] = dat_obesity.processed_text.apply(lambda row: list(map(bigrams, row)))

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'ngrams' is not defined

In [17]:
dat_obesity.head()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,index,abnormal hormone,abnormal level,abuse poor,acquire secondary,alcohol consumption,alcohol drug,also help,anxiety alcohol,...,Dash,Quote,Apostro,Parenth,OtherP,processed_text,processed_text_bert,tokens,final_text,tokens_bigrams
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.33,0.0,0.00,0.00,0.00,a morbidly obese baby has baffled doctors by...,A morbidly obese baby has baffled doctors - by...,"['morbidly', 'obese', 'baby', 'baffle', 'docto...",morbidly obese baby baffle doctor weigh whop lbs,<function <lambda> at 0x7fd586254280>
1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,this dachshund weighed pounds when he was resc...,This dachshund weighed 77 pounds when he was r...,"['dachshund', 'weigh', 'pound', 'rescue', 'look']",dachshund weigh pound rescue look,<function <lambda> at 0x7fd586254280>
2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.31,0.0,1.84,1.05,2.89,now ask yourself why me jiggling my butt fat ...,"Now, ask yourself why me jiggling my butt fat ...","['ask', 'jiggle', 'butt', 'fat', 'make', 'feel...",ask jiggle butt fat make feel weird youve sort...,<function <lambda> at 0x7fd586254280>
3,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,2.78,an obese baby and a young girl with a rare dis...,An obese baby and a young girl with a rare dis...,"['obese', 'baby', 'young', 'girl', 'rare', 'di...",obese baby young girl rare disease whose dream...,<function <lambda> at 0x7fd586254280>
4,4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.72,0.0,4.56,0.00,0.24,according to some men i am going to die young ...,According to some men I am going to die young ...,"['accord', 'men', 'go', 'die', 'young', 'obesi...",accord men go die young obesity make bad healt...,<function <lambda> at 0x7fd586254280>
